## Recommender System with LF:

In [86]:
from scipy import sparse
import pandas as pd
import numpy as np
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity
import requests
import json

In [87]:
def get_Merged_Data(Affiliates,Commissions,Banners):
    AffiliatesCommissions= pd.merge(Affiliates, Commissions, on='userid',how='inner')
    final= pd.merge(AffiliatesCommissions, Banners,left_on='bannerid', right_on='id',how='inner')
    final=final.drop(['id','id_x','name'], axis=1)
    return final

In [88]:
def import_data():
    affiliates=pd.read_csv("affiliates.csv")
    commissions=pd.read_csv("all_commissions.csv")
    banners=pd.read_csv("banners.csv")
    final=get_Merged_Data(Affiliates=affiliates,Banners=banners,Commissions=commissions)
    final = final.drop(['username','data4','salesCount','commissions','firstname', 'lastname','campaignname', 'dateinserted_x', 'data9', 'data10','data11',"dateinserted_y", 'id_y','rstatus_x', 'rtype',  'rstatus_y', 'description', 'account', 'category'],axis=1)
    return final

In [89]:
### use banner_dict Function to create an banner dictionary based on their banner_id and banners name Required Input
def create_banner_dict(final,id_col,name_col):
    df = pd.DataFrame(final, columns = [id_col,name_col])
    df=df.drop_duplicates()
    df=df.reset_index();
    banner_dict ={}
    for i in range(df.shape[0]):
        banner_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return banner_dict

In [90]:
## interaction_matrix where rows represent each user id and columns represent each banner id with commissions as values. 
def create_interaction_matrix(df,user_col, banner_col, commission_col, norm= False, threshold = None):
    interactions = df.groupby([user_col, banner_col])[commission_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

In [91]:
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

In [92]:
####  Building the light Matrix Factorization model
def runMF(interactions, n_components=30, loss='warp', k=10, epoch=30,n_jobs = 4):
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

In [93]:
#mf_model = runMF(interactions = interactions,n_components = 30,loss = 'warp',epoch = 70 ,n_jobs = 4)

## Banner recommendation to banners 
recommend a list of banners specific to a particular banner

In [94]:
#### cosine distance matrix using banner embeddings generated by matrix factorization model. This will help us calculate similarity between banners, and then we can recommend top N similar banners to a banner of interest.  
def create_banner_emdedding_distance_matrix(final):
    interactions= create_interaction_matrix(df = final,user_col = 'userid',banner_col = 'bannerid',commission_col = 'commission')
    model=runMF(interactions=interactions,n_components = 30,loss = 'warp',epoch = 70 ,n_jobs = 4)
    df_banner_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_banner_norm_sparse)
    banner_emdedding_distance_matrix = pd.DataFrame(similarities)
    banner_emdedding_distance_matrix.columns = interactions.columns
    banner_emdedding_distance_matrix.index = interactions.columns
    return banner_emdedding_distance_matrix

In [95]:
def banner_banner_recommendation(banner_id, n_banners = 10, show = False):
    final=import_data()
    banner_emdedding_distance_matrix=create_banner_emdedding_distance_matrix(final)
    banner_dict=create_banner_dict(final=final,id_col="bannerid",name_col="banner")
    recommended_banners = list(pd.Series(banner_emdedding_distance_matrix.loc[banner_id,:]. \
                                  sort_values(ascending = False).head(n_banners+1). \
                                  index[1:n_banners+1]))
    if show == False:
        print("banner of interest :{0}".format(banner_dict[banner_id]))
        print("banner similar to the above banner:")
        counter = 1
        for i in recommended_banners:
            print(str(counter) + '- ' +  banner_dict[i])
            counter+=1
    
    return recommended_banners

In [96]:
#banner_banner_recommendation('0c806e2d')

### Banner recommendation to a user 

In [97]:
def sample_recommendation_user(user_id,threshold = 0,nrec_banner = 5, show = False):
    final=import_data()
    interactions= create_interaction_matrix(df = final,user_col = 'userid',banner_col = 'bannerid',commission_col = 'commission')
    user_dict=create_user_dict(interactions=interactions)
    banner_dict=create_banner_dict(final=final,id_col="bannerid",name_col="banner")
    model=runMF(interactions=interactions,n_components = 30,loss = 'warp',epoch = 70 ,n_jobs = 4)
    
    n_users, n_banner = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_banner)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    known_banners = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
                                 .sort_values(ascending=False))
    scores = [x for x in scores if x not in known_banners]
    return_score_list = scores[0:nrec_banner]
    known_banners = list(pd.Series(known_banners).apply(lambda x: banner_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: banner_dict[x]))
    if show == False:
        print("Known Likes banners:")
        counter = 1
        for i in known_banners:
            print(str(counter) + '* ' + i)
            counter+=1
        print("\n Recommended banners which user hopefully will be interested in:")
        counter = 1
        for i in scores:
            print(str(counter) + '* ' + i)
            counter+=1
    
    return return_score_list


In [98]:
#sample_recommendation_user(user_id = '04042b26',threshold = 0.5,nrec_banner = 5)


In [99]:
#rec_list

###  User recommendation to a item

In [103]:
####  recommend a list of users specific to a particular banner
def sample_recommendation_user_to_item(banner_id,number_of_user):
    final=import_data()
    interactions= create_interaction_matrix(df = final,user_col = 'userid',banner_col = 'bannerid',commission_col = 'commission')
    user_dict=create_user_dict(interactions=interactions)
    banner_dict=create_banner_dict(final=final,id_col="bannerid",name_col="banner")
    model=runMF(interactions=interactions,n_components = 30,loss = 'warp',epoch = 70 ,n_jobs = 4)
    n_users, n_items = interactions.shape
    banner_id = banner_dict[banner_id]
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users),np.repeat(x.searchsorted(banner_id), n_users)))
    user_list = list(interactions.index[scores.sort_values(
        ascending=False).head(number_of_user).index])
    return user_list

In [107]:

## Calling 10 user recommendation for banner_id 
#sample_recommendation_user_to_item(banner_id = '1eb1e0e0',number_of_user = 10)